In [1]:
# Load environment variables and set up auto-reload
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload

In [2]:
import sys
sys.path.append('../')

In [3]:
%%writefile ../src/full_agent.py

"""This code contains the code for the logistician agent"""

import os
from langchain_core.messages import HumanMessage, AIMessage, get_buffer_string
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

from src.supervisor_schema import AgentInputState
from src.supervisor_agent import DelegateNextAgent, clarify_with_user, supervisor_agent, supervisor_tools
from src.logistics_agent import LogisticsAgent , LogisticsState
from src.forwarder_agent import ForwarderAgent

# Build the scoping workflow
full_agent_builder = StateGraph(LogisticsState, input_schema=AgentInputState)

# Add workflow nodes
full_agent_builder.add_node("supervisor_agent"  , supervisor_agent)
full_agent_builder.add_node("supervisor_tools"  , supervisor_tools)
full_agent_builder.add_node("clarify_with_user" , clarify_with_user)
full_agent_builder.add_node("LogisticsAgent"    , LogisticsAgent)
full_agent_builder.add_node("ForwarderAgent"    , ForwarderAgent)

# Add workflow edges
full_agent_builder.add_edge(START, "supervisor_agent")
full_agent_builder.add_conditional_edges(
    "supervisor_agent",
     DelegateNextAgent,
    {
        "supervisor_tools"  : "supervisor_tools" , 
        "clarify_with_user" : "clarify_with_user", 
        "logistics_agent"   : "LogisticsAgent"   ,
        "forwarder_agent"   : "ForwarderAgent"
    },
)
full_agent_builder.add_edge("supervisor_tools", "supervisor_agent")
full_agent_builder.add_edge("clarify_with_user", END)

# Compile the workflow
checkpointer = InMemorySaver()
full_agent = full_agent_builder.compile(checkpointer = checkpointer)

Overwriting ../src/full_agent.py


In [4]:
from IPython.display import Image, display
from src.full_agent import full_agent

# Show the agent
display(Image(full_agent.get_graph(xray=True).draw_mermaid_png()))

c:\Users\kamalmo\Desktop\AI\InboundLogisticSystem\MultiAgentInboundLogistic\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 429.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [5]:
from src.utils import format_message
from langchain_core.messages import HumanMessage
from src.full_agent import full_agent

thread = {"configurable":{"thread_id":"1"}}
result = await full_agent.ainvoke({"messages":[HumanMessage(content="I want to enter the AWB/BL 12345")]} , config=thread)
format_message(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the AWB/BL 12345                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the AWB/BL number '12345', which is a required field for the logistics department.        │
│ Assigning this task to the Logistics Agent for further handling and data entry.                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Fields**                                                                                   │
│                                                                                                                 │
│ Your request is missing the following required information:                                                     │
│ - AWB/BL Date                                                                                                   │
│ - Shipment Mode                                                                                                 │
│                                                                                                                 │
│ Please provide these details so we can proceed. If you need help or clarification on what these fields mean,    │
│ let me know!                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [6]:
result = await full_agent.ainvoke({"messages":[HumanMessage(content="Skip")]} , config=thread)
format_message(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the AWB/BL 12345                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the AWB/BL number '12345', which is a required field for the logistics department.        │
│ Assigning this task to the Logistics Agent for further handling and data entry.                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Fields**                                                                                   │
│                                                                                                                 │
│ Your request is missing the following required information:                                                     │
│ - AWB/BL Date                                                                                                   │
│ - Shipment Mode                                                                                                 │
│                                                                                                                 │
│ Please provide these details so we can proceed. If you need help or clarification on what these fields mean,    │
│ let me know!                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Skip                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ You have chosen to skip providing the required fields 'AWB/BL Date' and 'Shipment Mode'. Would you like to      │
│ proceed without entering these details, or do you want to provide them now? Please confirm how you wish to      │
│ continue.                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [7]:
result = await full_agent.ainvoke({"messages":[HumanMessage(content="The Shipment Mode is Air")]} , config=thread)
format_message(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the AWB/BL 12345                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the AWB/BL number '12345', which is a required field for the logistics department.        │
│ Assigning this task to the Logistics Agent for further handling and data entry.                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Fields**                                                                                   │
│                                                                                                                 │
│ Your request is missing the following required information:                                                     │
│ - AWB/BL Date                                                                                                   │
│ - Shipment Mode                                                                                                 │
│                                                                                                                 │
│ Please provide these details so we can proceed. If you need help or clarification on what these fields mean,    │
│ let me know!                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Skip                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ You have chosen to skip providing the required fields 'AWB/BL Date' and 'Shipment Mode'. Would you like to      │
│ proceed without entering these details, or do you want to provide them now? Please confirm how you wish to      │
│ continue.                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ The Shipment Mode is Air                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ You have provided the AWB/BL number '12345' and the Shipment Mode as 'Air', but the required field 'AWB/BL      │
│ Date' is still missing. Would you like to provide the AWB/BL Date now, or proceed without it?                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [8]:
result = await full_agent.ainvoke({"messages":[HumanMessage(content="proceed")]} , config=thread)
format_message(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the AWB/BL 12345                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the AWB/BL number '12345', which is a required field for the logistics department.        │
│ Assigning this task to the Logistics Agent for further handling and data entry.                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Fields**                                                                                   │
│                                                                                                                 │
│ Your request is missing the following required information:                                                     │
│ - AWB/BL Date                                                                                                   │
│ - Shipment Mode                                                                                                 │
│                                                                                                                 │
│ Please provide these details so we can proceed. If you need help or clarification on what these fields mean,    │
│ let me know!                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Skip                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ You have chosen to skip providing the required fields 'AWB/BL Date' and 'Shipment Mode'. Would you like to      │
│ proceed without entering these details, or do you want to provide them now? Please confirm how you wish to      │
│ continue.                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ The Shipment Mode is Air                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ You have provided the AWB/BL number '12345' and the Shipment Mode as 'Air', but the required field 'AWB/BL      │
│ Date' is still missing. Would you like to provide the AWB/BL Date now, or proceed without it?                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ proceed                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the following inbound logistics data: AWB/BL number '12345' and Shipment Mode 'Air'. The  │
│ user has explicitly chosen to proceed without providing the required 'AWB/BL Date'. Assigning this task to the  │
│ Logistics Agent for further handling, noting the missing AWB/BL Date as per the user's instruction.             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Field:**                                                                                   │
│ - **AWB/BL Date**                                                                                               │
│                                                                                                                 │
│ Please provide the missing information so we can proceed with your logistics request. If you need help or       │
│ clarification about this field, let me know!                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [9]:
result = await full_agent.ainvoke({"messages":[HumanMessage(content="AWB/BL Date is 2025-09-30")]} , config=thread)
format_message(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the AWB/BL 12345                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the AWB/BL number '12345', which is a required field for the logistics department.        │
│ Assigning this task to the Logistics Agent for further handling and data entry.                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Fields**                                                                                   │
│                                                                                                                 │
│ Your request is missing the following required information:                                                     │
│ - AWB/BL Date                                                                                                   │
│ - Shipment Mode                                                                                                 │
│                                                                                                                 │
│ Please provide these details so we can proceed. If you need help or clarification on what these fields mean,    │
│ let me know!                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Skip                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ You have chosen to skip providing the required fields 'AWB/BL Date' and 'Shipment Mode'. Would you like to      │
│ proceed without entering these details, or do you want to provide them now? Please confirm how you wish to      │
│ continue.                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the following inbound logistics data: AWB/BL number '12345' and Shipment Mode 'Air'. The  │
│ user has explicitly chosen to proceed without providing the required 'AWB/BL Date'. Assigning this task to the  │
│ Logistics Agent for further handling, noting the missing AWB/BL Date as per the user's instruction.             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Field:**                                                                                   │
│ - **AWB/BL Date**                                                                                               │
│                                                                                                                 │
│ Please provide the missing information so we can proceed with your logistics request. If you need help or       │
│ clarification about this field, let me know!                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ AWB/BL Date is 2025-09-30                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided all required inbound logistics data: AWB/BL number '12345', Shipment Mode 'Air', and      │
│ AWB/BL Date '2025-09-30'. No further clarification is needed. Assigning this task to the Logistics Agent for    │
│ processing.                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Some optional fields are missing from your submission: Forwarder, Product Temperature, Packing, Shipping Temp,  │
│ Gel Pack Expiry Date, Handover to Clearance, Aggregation, Notified FF Date, Green light - Date, Logistic        │
│ Comment, Remark, and ASN Importation Date.                                                                      │
│                                                                                                                 │
│ You can:                                                                                                        │
│ - Provide any of these details now for a more complete record,                                                  │
│ - Let me know to skip them and proceed,                                                                         │
│ - Or update them later if needed.                                                                               │
│                                                                                                                 │
│ How would you like to proceed?                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [10]:
result = await full_agent.ainvoke({"messages":[HumanMessage(content="skip")]} , config=thread)
format_message(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the AWB/BL 12345                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the AWB/BL number '12345', which is a required field for the logistics department.        │
│ Assigning this task to the Logistics Agent for further handling and data entry.                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Fields**                                                                                   │
│                                                                                                                 │
│ Your request is missing the following required information:                                                     │
│ - AWB/BL Date                                                                                                   │
│ - Shipment Mode                                                                                                 │
│                                                                                                                 │
│ Please provide these details so we can proceed. If you need help or clarification on what these fields mean,    │
│ let me know!                                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Skip                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ You have chosen to skip providing the required fields 'AWB/BL Date' and 'Shipment Mode'. Would you like to      │
│ proceed without entering these details, or do you want to provide them now? Please confirm how you wish to      │
│ continue.                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided all required inbound logistics data: AWB/BL number '12345', Shipment Mode 'Air', and      │
│ AWB/BL Date '2025-09-30'. No further clarification is needed. Assigning this task to the Logistics Agent for    │
│ processing.                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Some optional fields are missing from your submission: Forwarder, Product Temperature, Packing, Shipping Temp,  │
│ Gel Pack Expiry Date, Handover to Clearance, Aggregation, Notified FF Date, Green light - Date, Logistic        │
│ Comment, Remark, and ASN Importation Date.                                                                      │
│                                                                                                                 │
│ You can:                                                                                                        │
│ - Provide any of these details now for a more complete record,                                                  │
│ - Let me know to skip them and proceed,                                                                         │
│ - Or update them later if needed.                                                                               │
│                                                                                                                 │
│ How would you like to proceed?                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ skip                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ All required inbound logistics data has been provided: AWB/BL number '12345', Shipment Mode 'Air', and AWB/BL   │
│ Date '2025-09-30'. The user has chosen to skip all optional fields. Assigning this task to the Logistics Agent  │
│ for processing as per the user's instructions.                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Here is the final list of fields for your confirmation:                                                         │
│                                                                                                                 │
│ - AWB/BL: 12345                                                                                                 │
│ - AWB/BL Date: 2025-09-30                                                                                       │
│ - Shipment Mode: Air                                                                                            │
│                                                                                                                 │
│ The following optional fields are still missing:                                                                │
│ - Forwarder                                                                                                     │
│ - Product Temperature                                                                                           │
│ - Packing                                                                                                       │
│ - Shipping Temp                                                                                                 │
│ - Gel Pack Expiry Date                                                                                          │
│ - Handover to Clearance                                                                                         │
│ - Aggregation                                                                                                   │
│ - Notified FF Date                                                                                              │
│ - Green light - Date                                                                                            │
│ - Logistic Comment                                                                                              │
│ - Remark                                                                                                        │
│ - ASN Importation Date                                                                                          │
│                                                                                                                 │
│ No mandatory fields are missing.                                                                                │
│ Please confirm if the above information is correct, or let me know if you want to add any of the optional       │
│ fields before submission.                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
result = await full_agent.ainvoke({"messages":[HumanMessage(content="confirm")]} , config=thread)
format_message(result["messages"])

UnsupportedOperation: fileno

In [12]:
from src.utils import format_message
from langchain_core.messages import HumanMessage
from src.full_agent import full_agent

thread = {"configurable":{"thread_id":"2"}}
result = await full_agent.ainvoke({"messages":[HumanMessage(content="I want to enter the Shipment Readiness Date is 2025-09-30")]} , config=thread)
format_message(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the Shipment Readiness Date is 2025-09-30                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided a Shipment Readiness Date of 2025-09-30. This field is managed by the Forwarder Agent.    │
│ Assigning this task to the Forwarder Agent for further processing.                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Information**                                                                              │
│                                                                                                                 │
│ I cannot proceed with your Forwarder request because the following required field is missing:                   │
│                                                                                                                 │
│ - **Pick Up Date**                                                                                              │
│                                                                                                                 │
│ **How to provide the information:**                                                                             │
│ Please reply with the missing detail in a clear format, for example:                                            │
│ - Pick Up Date: 2024-07-15                                                                                      │
│                                                                                                                 │
│ If you need help or clarification about this field, let me know!                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [13]:
result = await full_agent.ainvoke({"messages":[HumanMessage(content="I want to enter the Shipment Readiness Date is 2025-09-30 and nd Pick up date is 2025-08-30")]} , config=thread)
format_message(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the Shipment Readiness Date is 2025-09-30                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided a Shipment Readiness Date of 2025-09-30. This field is managed by the Forwarder Agent.    │
│ Assigning this task to the Forwarder Agent for further processing.                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Information**                                                                              │
│                                                                                                                 │
│ I cannot proceed with your Forwarder request because the following required field is missing:                   │
│                                                                                                                 │
│ - **Pick Up Date**                                                                                              │
│                                                                                                                 │
│ **How to provide the information:**                                                                             │
│ Please reply with the missing detail in a clear format, for example:                                            │
│ - Pick Up Date: 2024-07-15                                                                                      │
│                                                                                                                 │
│ If you need help or clarification about this field, let me know!                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the Shipment Readiness Date is 2025-09-30 and nd Pick up date is 2025-08-30                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the following details: Shipment Readiness Date: 2025-09-30 and Pick Up Date: 2025-08-30.  │
│ Both fields are managed by the Forwarder Agent. Assigning this task to the Forwarder Agent for further          │
│ processing.                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Some optional fields are missing from your request:                                                             │
│ ['No. of Pallets', 'No.of Containers', 'Commodity Description', 'Country Of Origin', 'AirPort/SeaPort name',    │
│ 'Shipping Line/Airline', 'Port Of Destination', 'Actual ATA - FF', 'Gross weight (KG)', 'CBM', 'ETD',           │
│ 'Chargable Weight(KG)', 'Freight Cost', 'Total values of Goods', 'Freight Invoice Number', 'ETA', 'Comment']    │
│                                                                                                                 │
│ You can provide any of these details now to make your record more complete, or let me know if you'd like to     │
│ skip them and proceed. If you prefer, you can also add this information later. Please let me know how you'd     │
│ like to continue!                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [14]:
result = await full_agent.ainvoke({"messages":[HumanMessage(content="skip")]} , config=thread)
format_message(result["messages"])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the Shipment Readiness Date is 2025-09-30                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided a Shipment Readiness Date of 2025-09-30. This field is managed by the Forwarder Agent.    │
│ Assigning this task to the Forwarder Agent for further processing.                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ ⚠️ **Missing Required Information**                                                                              │
│                                                                                                                 │
│ I cannot proceed with your Forwarder request because the following required field is missing:                   │
│                                                                                                                 │
│ - **Pick Up Date**                                                                                              │
│                                                                                                                 │
│ **How to provide the information:**                                                                             │
│ Please reply with the missing detail in a clear format, for example:                                            │
│ - Pick Up Date: 2024-07-15                                                                                      │
│                                                                                                                 │
│ If you need help or clarification about this field, let me know!                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to enter the Shipment Readiness Date is 2025-09-30 and nd Pick up date is 2025-08-30                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the following details: Shipment Readiness Date: 2025-09-30 and Pick Up Date: 2025-08-30.  │
│ Both fields are managed by the Forwarder Agent. Assigning this task to the Forwarder Agent for further          │
│ processing.                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Some optional fields are missing from your request:                                                             │
│ ['No. of Pallets', 'No.of Containers', 'Commodity Description', 'Country Of Origin', 'AirPort/SeaPort name',    │
│ 'Shipping Line/Airline', 'Port Of Destination', 'Actual ATA - FF', 'Gross weight (KG)', 'CBM', 'ETD',           │
│ 'Chargable Weight(KG)', 'Freight Cost', 'Total values of Goods', 'Freight Invoice Number', 'ETA', 'Comment']    │
│                                                                                                                 │
│ You can provide any of these details now to make your record more complete, or let me know if you'd like to     │
│ skip them and proceed. If you prefer, you can also add this information later. Please let me know how you'd     │
│ like to continue!                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ skip                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ The user has provided the following details: Shipment Readiness Date: 2025-09-30 and Pick Up Date: 2025-08-30.  │
│ The user has chosen to skip all optional fields at this time. Assigning this task to the Forwarder Agent for    │
│ further handling.                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Here is the final list of fields for your confirmation:                                                         │
│                                                                                                                 │
│ **Provided:**                                                                                                   │
│ - Shipment Readiness Date: 2025-09-30                                                                           │
│ - Pick Up Date: 2025-08-30                                                                                      │
│                                                                                                                 │
│ **Missing (optional):**                                                                                         │
│ - No. of Pallets                                                                                                │
│ - No. of Containers                                                                                             │
│ - Commodity Description                                                                                         │
│ - Country Of Origin                                                                                             │
│ - AirPort/SeaPort name                                                                                          │
│ - Shipping Line/Airline                                                                                         │
│ - Port Of Destination                                                                                           │
│ - Actual ATA - FF                                                                                               │
│ - Gross weight (KG)                                                                                             │
│ - CBM                                                                                                           │
│ - ETD                                                                                                           │
│ - Chargable Weight (KG)                                                                                         │
│ - Freight Cost                                                                                                  │
│ - Total value of Goods                                                                                          │
│ - Freight Invoice Number                                                                                        │
│ - ETA                                                                                                           │
│ - Comment                                                                                                       │
│                                                                                                                 │
│ Please confirm if the above information is correct and if you wish to proceed.                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [15]:
result = await full_agent.ainvoke({"messages":[HumanMessage(content="confirm")]} , config=thread)
format_message(result["messages"])

UnsupportedOperation: fileno